# Policy Evaluation

**Author:** ZHENG Wenjie

**Last Update:** 2021-08-25

This notebook is related to Section 4.1 of the book. It studies the policy evaluation problem for the GridWorld structure. In particular, it implements three methods: directly solving the linear system, fixed-point iteration, and in-place fixed-point iteration.

The theoretic guarantee of the convergence of the algorithms is thanks to the compressing mapping theorem. The transition matrix is a stochastic matrix (i.e., the row sums equal to 1 => the largest eigenvalue is 1), and it is then multiplied by $\gamma<1$.

In [1]:
import numpy as np

In [2]:
class Maze:
    def __init__(self, m, n, exit=None, teleport=None, reward=None):
        self.m = m
        self.n = n
        self.exit = exit
        self.link = np.zeros((m*n, 4), dtype=int)
        for i in range(m*n):
            if teleport is None or i not in teleport:
                j = i - n
                j = i if j<0 else j
                self.link[i, 0] = j
                j = i - 1
                j = i if j%n==n-1 else j
                self.link[i, 1] = j
                j = i + 1
                j = i if j%n==0 else j
                self.link[i, 2] = j
                j = i + n
                j = i if j>=m*n else j
                self.link[i, 3] = j
            else:
                self.link[i, :] = teleport[i]
                
        self.reward = reward
        
    def evaluate(self, γ, π=None, direct=True, inplace=False, n_iter=60):
        m, n = self.m, self.n
        if π is None:
            π = np.full((m*n, 4), 0.25)

        if direct or not inplace:
            A = np.zeros((m*n, m*n))
            for i in range(m*n):
                for j in range(4):
                    A[i, self.link[i, j]] += π[i, j]
            b = np.sum(π * self.reward, axis=1)
            
        if direct:
            return np.linalg.solve(np.eye(m*n)-γ*A, b).reshape(m, n)
        
        if not inplace:
            trace = [np.zeros(m*n)]
            for _ in range(n_iter):
                trace.append(γ * np.dot(A, trace[-1]) + b)
            return trace
        else:
            trace = []
            v = np.zeros(m*n)
            for _ in range(n_iter):
                for i in range(m*n):
                    v[i] = sum(π[i, :] * (self.reward[i, :] + γ*v[self.link[i, :]]))
                    trace.append(v[i])
            return v, trace

In [3]:
reward = np.zeros((5*5, 4))
for i in range(5):
    reward[i, 0] = -1
for i in range(0, 5*5, 5):
    reward[i, 1] = -1
for i in range(4, 5*5, 5):
    reward[i, 2] = -1
for i in range(20, 5*5):
    reward[i, 3] = -1
reward[1, :] = 10
reward[3, :] = 5

maze1 = Maze(5, 5, teleport={1:21, 3:13}, reward=reward)

In [4]:
maze1.evaluate(0.9)

array([[ 3.30899634,  8.78929186,  4.42761918,  5.32236759,  1.49217876],
       [ 1.52158807,  2.99231786,  2.25013995,  1.9075717 ,  0.54740271],
       [ 0.05082249,  0.73817059,  0.67311326,  0.35818621, -0.40314114],
       [-0.9735923 , -0.43549543, -0.35488227, -0.58560509, -1.18307508],
       [-1.85770055, -1.34523126, -1.22926726, -1.42291815, -1.97517905]])

In [5]:
maze1.evaluate(0.9, direct=False)[-1]

array([ 3.30919888,  8.78949409,  4.4278208 ,  5.3225686 ,  1.49237943,
        1.52179059,  2.99252004,  2.25034156,  1.90777275,  0.5476034 ,
        0.051025  ,  0.73837276,  0.67331488,  0.35838728, -0.40294041,
       -0.97338981, -0.43529326, -0.35468064, -0.585404  , -1.18287432,
       -1.85749806, -1.3450291 , -1.22906562, -1.42271704, -1.97497827])

In [6]:
maze1.evaluate(0.9, direct=False, inplace=True)[0]

array([ 3.30901641,  8.78930766,  4.42763437,  5.32238134,  1.49219316,
        1.52160654,  2.99233348,  2.25015413,  1.90758496,  0.54741581,
        0.05084012,  0.73818574,  0.67312688,  0.35819896, -0.40312873,
       -0.97357513, -0.43548062, -0.35486898, -0.58559268, -1.18306305,
       -1.85768358, -1.34521662, -1.22925413, -1.4229059 , -1.97516718])